In [5]:
import sys
import os
sys.path.append('..')
from utils import load_data, nx2homg
import networkx as nx
import numpy as np
from random import shuffle

In [11]:
from homlib import hom

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn import svm
from sklearn.metrics import f1_score, accuracy_score
from sklearn.neural_network import MLPClassifier

In [29]:
import pickle as pkl

In [98]:
from sklearn.preprocessing import StandardScaler

# Load REDDIT-BINARY

In [3]:
reddit_graphs = load_data("REDDITBINARY", degree_as_tag=False)

loading data
# classes: 2
# maximum node tag: 1
# data: 2000


# Test counting trees

In [7]:
test_g = nx2homg(reddit_graphs[0][0].g)

In [8]:
trees_5 = [nx2homg(f) for f in nx.generators.nonisomorphic_trees(5)]

In [9]:
len(trees_5)

3

In [11]:
%%time
hom(trees_5[0], test_g)

CPU times: user 29.2 s, sys: 29.6 ms, total: 29.3 s
Wall time: 29.2 s


([<Graph object, undirected, with 5 vertices and 4 edges at 0x7f70415f2b70>,
  <Graph object, undirected, with 5 vertices and 4 edges at 0x7f70415f2a20>,
  <Graph object, undirected, with 5 vertices and 4 edges at 0x7f70415f2940>],
 [10622826, 491335, 8971])

In [12]:
%%time
hom(trees_5[0], test_g)

CPU times: user 53 ms, sys: 3.32 ms, total: 56.4 ms
Wall time: 56.2 ms


2208660.0

In [13]:
%%time
hom(trees_5[1], test_g)

CPU times: user 54 ms, sys: 28 µs, total: 54 ms
Wall time: 53.9 ms


3395568.0

In [14]:
%%time
hom(trees_5[2], test_g)

CPU times: user 53.4 ms, sys: 3 µs, total: 53.4 ms
Wall time: 53.3 ms


268535990.0

In [16]:
trees_6 = [nx2homg(f) for f in nx.generators.nonisomorphic_trees(6)]

In [17]:
len(trees_6)

6

In [19]:
%%time
for t in trees_6:
    print(hom(t, test_g))

7135750.0
272452198.0
24076378.0
8837568.0
420470932.0
34360930080.0
CPU times: user 315 ms, sys: 3.38 ms, total: 318 ms
Wall time: 317 ms


# Build Homomorphism profile

In [20]:
trees_2 = [nx2homg(f) for f in nx.generators.nonisomorphic_trees(2)] 
trees_3 = [nx2homg(f) for f in nx.generators.nonisomorphic_trees(3)] 
trees_4 = [nx2homg(f) for f in nx.generators.nonisomorphic_trees(4)] 
trees_5 = [nx2homg(f) for f in nx.generators.nonisomorphic_trees(5)] 
trees_6 = [nx2homg(f) for f in nx.generators.nonisomorphic_trees(6)] 
trees_7 = [nx2homg(f) for f in nx.generators.nonisomorphic_trees(7)] 
trees_8 = [nx2homg(f) for f in nx.generators.nonisomorphic_trees(8)] 
trees_9 = [nx2homg(f) for f in nx.generators.nonisomorphic_trees(9)] 

In [24]:
all_trees = (*trees_2, *trees_3, *trees_4, *trees_5, *trees_6)

In [25]:
len(all_trees)

13

In [22]:
len(reddit_graphs[0])

2000

In [26]:
%%time
# This one run in 3 hrs on a single thread
X = []
y = []  
for g in reddit_graphs[0]:
    gh = nx2homg(g.g)
    profile = [hom(t, gh) for t in all_trees]
    X.append(profile)
    y.append(g.label)

CPU times: user 2h 51min 8s, sys: 2min 41s, total: 2h 53min 49s
Wall time: 2h 53min 56s


In [31]:
# Dump X so we don't have to run for Reddit again
with open('REDDITBINARY_TREE6.pkl', 'wb') as f:
    pkl.dump(X, f)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)
normalizer = Normalizer().fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)

In [72]:
%%time
clf = svm.SVC(C=1000.0, gamma=1.0)
clf.fit(X_train, y_train)

print("Test acc: ", f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="micro"))

Test acc:  0.775
CPU times: user 92.2 ms, sys: 0 ns, total: 92.2 ms
Wall time: 91 ms


In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
normalizer = Normalizer().fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)

In [96]:
%%time
clf = MLPClassifier(hidden_layer_sizes=(32,32), activation='relu', 
                    solver='adam', alpha=0.000001, learning_rate='adaptive', learning_rate_init=0.03)
clf.fit(X_train, y_train)
print("Test acc: ", f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="micro"))

Test acc:  0.785
CPU times: user 1.44 s, sys: 6.57 ms, total: 1.45 s
Wall time: 364 ms


# use scaler

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)
normalizer = StandardScaler().fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)

In [132]:
%%time
clf = MLPClassifier(hidden_layer_sizes=(32,16), activation='relu', 
                    solver='adam', alpha=0.00001, learning_rate='adaptive', learning_rate_init=0.03)
clf.fit(X_train, y_train)
print("Test acc: ", f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="macro"))

Test acc:  0.8146431881371641
CPU times: user 2.5 s, sys: 13.3 ms, total: 2.51 s
Wall time: 630 ms


# Run 10 fold 

In [211]:
%%time
mic = []
mac = []
for i in range(10):
    seed = np.random.randint(0,1000)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)
    normalizer = StandardScaler().fit(X_train)
    X_train = normalizer.transform(X_train)
    X_test = normalizer.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes=(32,32), activation='relu', batch_size=64, 
                        solver='sgd', alpha=0.00001, learning_rate_init=0.01, early_stopping=True)
    clf.fit(X_train, y_train)
    print("Test acc: ", i, f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="micro"))
    mic.append(f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="micro"))
    mac.append(f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="macro"))
print(np.mean(mic), np.std(mic))
print(np.mean(mac), np.std(mac))

Test acc:  0 0.735
Test acc:  1 0.78
Test acc:  2 0.75
Test acc:  3 0.735
Test acc:  4 0.745
Test acc:  5 0.82
Test acc:  6 0.775
Test acc:  7 0.765
Test acc:  8 0.775
Test acc:  9 0.715
0.7595000000000001 0.028412145290350747
0.7573134194172135 0.028042406101482877
CPU times: user 7.32 s, sys: 99.9 ms, total: 7.41 s
Wall time: 1.87 s


In [ ]:
%%time
mic = []
mac = []
for i in range(10):
    seed = np.random.randint(0,1000)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)
    normalizer = StandardScaler().fit(X_train)
    X_train = normalizer.transform(X_train)
    X_test = normalizer.transform(X_test)
    clf = svm.SVC(C=400.0, gamma=50.0, kernel='poly')
    clf.fit(X_train, y_train)
    print("Test acc: ", i, f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="micro"))
    mic.append(f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="micro"))
    mac.append(f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="macro"))
print(np.mean(mic), np.std(mic))
print(np.mean(mac), np.std(mac))

# K-neighbors

In [240]:
from sklearn.neighbors import KNeighborsClassifier

In [246]:
%%time
mic = []
mac = []
for i in range(10):
    seed = np.random.randint(0,1000)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)
    normalizer = StandardScaler().fit(X_train)
    X_train = normalizer.transform(X_train)
    X_test = normalizer.transform(X_test)
    clf = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', 
                     leaf_size=30, p=8, metric='minkowski', 
                     metric_params=None, n_jobs=None)
    clf.fit(X_train, y_train)
    print("Test acc: ", i, f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="micro"))
    mic.append(f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="micro"))
    mac.append(f1_score(y_pred=clf.predict(X_test), y_true=y_test, average="macro"))
print(np.mean(mic), np.std(mic))
print(np.mean(mac), np.std(mac))

Test acc:  0 0.855
Test acc:  1 0.845
Test acc:  2 0.845
Test acc:  3 0.855
Test acc:  4 0.795
Test acc:  5 0.8399999999999999
Test acc:  6 0.795
Test acc:  7 0.835
Test acc:  8 0.8399999999999999
Test acc:  9 0.87
0.8374999999999998 0.023264780248263662
0.8366009623068489 0.02414167009872458
CPU times: user 423 ms, sys: 67 µs, total: 423 ms
Wall time: 419 ms
